In [30]:
import pandas as pd
import csv
from time import sleep
import requests as rq
import json
import os

data = pd.read_csv('crawl.csv')
data.dtypes

defi              object
logci_contract    object
dtype: object

In [31]:
for aaa, bbb in zip(data["defi"], data["logci_contract"]):
    print(aaa, bbb)

bzx_dai 0x1e9682d8c780790a5d5a9cb234634c8cc849e0b5
bzx_usdc 0x1e9682d8c780790a5d5a9cb234634c8cc849e0b5
bzx_eth 0xde744d544a9d768e96c21b5f087fc54b776e9b25
visor 0xC9f27A50f82571C1C8423A42970613b8dBDA14ef
umbrella 0xB3FB1D01B07A706736Ca175f827e4F56021b85dE
akropolis 0x1ceC0e358f882733c5ecC028d8A0c24Baee02004
value_defi 0x8764f2c305b79680cfcc3398a96aedea9260f7ff
ousd 0x328d15f6b5eba1c30cde1a5f1f5a9e35b07f5424
sanshu_inu 0x35c674c288577df3e9b5dafef945795b741c7810
harvest_finance 0x9b3be0cc5dd26fd0254088d03d8206792715588b
harvest_finance1 0xf8ce90c2710713552fb564869694b2505bfc0846
lendf_me 0x0eEe3E3828A45f7601D5F54bF49bB01d1A9dF5ea
yearn_finance_dai 0xACd43E627e64355f1861cEC6d3a6688B31a6F952
rari 0xce346ad7046222fd119f6b715b6f60c0fdbfe796
punk_dai 0x929cb86046E421abF7e1e02dE7836742654D49d6
punk_usdc 0x3BC6aA2D25313ad794b2D67f83f21D341cc3f5fb
punk_usdt 0x3BC6aA2D25313ad794b2D67f83f21D341cc3f5fb
popsicle_usdt_weth 0xc4ff55a4329f84f9Bf0F5619998aB570481EBB48
popsicle_usdc_weth 0xd63b340F6e9CCcF

In [32]:
def make_dir(path):
    folders = []
    while not os.path.isdir(path):
        path, suffix = os.path.split(path)
        folders.append(suffix)
    for folder in folders[::-1]:
        path = os.path.join(path, folder)
        os.mkdir(path)

def is_json(myjson):
    try:
        json_object = json.loads(myjson)
    except ValueError as e:
        return False
    return True

def is_nft(source):
    keywords = 'nft,NFT,ERC721,IERC721'
    return any([w in source and w for w in keywords.split(',')])

send_headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
    "Connection": "keep-alive",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "zh-CN,zh;q=0.9"}

contract_data = []

def download_url(url, retry=5, retry_delay=5, throw_if_fail=False):
    resp = rq.get(url, headers=send_headers, allow_redirects=False)

    if resp.status_code != 200:
        if retry > 0:
            sleep(retry_delay)
            f'Download source failed for {url}, status {resp.status_code}, retry in {retry_delay} secs'
            return download_url(url, retry-1, retry_delay)
        else:
            if throw_if_fail:
                raise Exception(f'Download source abort for {url}, status {resp.status_code}')
            return
    return resp

for contract_name, contract_address in zip(data["defi"], data["logci_contract"]):
    make_dir("crawl/"+contract_name)
    root = "./crawl/"+contract_name+"/"

    abi_link = 'https://api.etherscan.io/api?module=contract&action=getabi&address=' + \
                    contract_address+'&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW'
    abi_output = download_url(abi_link)
    abi_json_res = abi_output.json()
    abi_json = abi_json_res["result"]
    abi_file = open(root+"abi.json", 'w', encoding='UTF-8')
    abi_file.write(abi_json)
    # with open(root+contract_address+"/"+"abi.json", 'w+') as f:
    #     json.dump(abi_json, f)



    curl_link = 'https://api.etherscan.io/api?module=contract&action=getsourcecode&address=' + \
                    contract_address+'&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW'
    print(curl_link)
    output = download_url(curl_link)
    json_res = output.json()

    # json_data = json_res["result"]
    # print(json_data)
    json_file = open(root+"outputJson.json", 'w', encoding='UTF-8')
    # json_file.write(json_data)
    with open(root+"outputJson.json", 'w') as f:
        json.dump(json_res, f)

    if "result" in json_res:
        source_code = json_res["result"][0]["SourceCode"]
        # if not is_nft(source_code):
        #     continue
        make_dir(root+contract_address)
        if is_json(source_code):
            res = json.loads(source_code)
            for key in res:
                sol_file = open(
                    root+contract_address+"/"+key, 'w', encoding='UTF-8')
                sol_file.write(res[key]["content"])
                if "contract "+contract_name in res[key]["content"]:
                    contract_data_element = [contract_address, contract_name, key]
                    # contract_data.append(contract_data_element)
                    df = pd.DataFrame([contract_data_element],columns=['ContractAddress','ContractName','ContractPath'])
                    df.to_csv('./contract.csv',sep=',',mode='a',index=False,header=None)
                sol_file.close()
        elif source_code[0] == source_code[1] == "{":
            new_code = source_code[1:-1]
            res = json.loads(new_code)
            sources = res["sources"]
            for name in sources:
                # print(name)

                _dir, _file = os.path.split(
                    root+contract_address+"/"+name)
                # print(_dir)
                make_dir(_dir)
                sol_file = open(
                    root+contract_address+"/"+name, 'w', encoding='UTF-8')
                sol_file.write(sources[name]["content"])
                
                if "contract "+contract_name in sources[name]["content"]:
                    contract_data_element = [contract_address, contract_name, name]
                    # contract_data.append(contract_data_element)
                    df = pd.DataFrame([contract_data_element],columns=['ContractAddress','ContractName','ContractPath'])
                    df.to_csv('./contract.csv',sep=',',mode='a',index=False,header=None)
                sol_file.close()
        else:
            sol_file = open(root+contract_address + "/" +
                        contract_name + ".sol", 'w', encoding='UTF-8')
            sol_file.write(source_code)
            contract_data_element = [contract_address, contract_name, contract_name]
            df = pd.DataFrame([contract_data_element],columns=['ContractAddress','ContractName','ContractPath'])
            df.to_csv('./contract.csv',sep=',',mode='a',index=False,header=None)
            sol_file.close()

https://api.etherscan.io/api?module=contract&action=getsourcecode&address=0x1e9682d8c780790a5d5a9cb234634c8cc849e0b5&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
https://api.etherscan.io/api?module=contract&action=getsourcecode&address=0x1e9682d8c780790a5d5a9cb234634c8cc849e0b5&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
https://api.etherscan.io/api?module=contract&action=getsourcecode&address=0xde744d544a9d768e96c21b5f087fc54b776e9b25&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
https://api.etherscan.io/api?module=contract&action=getsourcecode&address=0xC9f27A50f82571C1C8423A42970613b8dBDA14ef&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
https://api.etherscan.io/api?module=contract&action=getsourcecode&address=0xB3FB1D01B07A706736Ca175f827e4F56021b85dE&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
https://api.etherscan.io/api?module=contract&action=getsourcecode&address=0x1ceC0e358f882733c5ecC028d8A0c24Baee02004&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
https://api.etherscan.io/api?module=contract&a

IndexError: string index out of range